In [1]:
import pandas as pd 
import numpy as np
from scipy import sparse
from scipy.optimize import minimize
from sklearn import linear_model
from datetime import timedelta
from pickle import load,dump

In [2]:
user_data = pd.read_csv('../data/micro/user_data-1k.csv')
cascade = pd.read_csv('../data/micro/diffusion-1k.csv',
                          parse_dates=['time'],
                          infer_datetime_format=True)
cascade.columns = ['original_status_id', 'original_user_id', 'time', 'user_id']  # DataFrame

In [16]:
k=100
ck = cascade.groupby(['original_status_id','original_user_id']).filter(lambda x:x.shape[0]>k) #result is not grouped 

In [17]:
ck

,original_status_id,original_user_id,time,user_id
21,3463921564867182,519514,2012-07-04 04:18:39,519514
22,3463921564867182,519514,2012-07-04 04:20:14,537924
23,3463921564867182,519514,2012-07-04 04:20:43,169868
24,3463921564867182,519514,2012-07-04 04:21:13,1490395
25,3463921564867182,519514,2012-07-04 04:21:48,537924
26,3463921564867182,519514,2012-07-04 04:22:18,1336687
27,3463921564867182,519514,2012-07-04 04:22:46,1205682
28,3463921564867182,519514,2012-07-04 04:22:55,1715509
29,3463921564867182,519514,2012-07-04 04:23:25,868627
30,3463921564867182,519514,2012-07-04 04:25:15,377807


In [6]:
s =30
pcascade = ck.sort_values(['time']).groupby(['original_status_id','original_user_id']).head(s)

In [7]:
pcascade

,original_status_id,original_user_id,time,user_id
21439,2011003225267462,1487992,2010-03-22 11:20:32,1487992
21440,2011003225267462,1487992,2010-03-22 14:38:07,242225
21441,2011003225267462,1487992,2010-03-22 18:04:18,509700
21442,2011003225267462,1487992,2010-03-22 20:21:34,1461926
21443,2011003225267462,1487992,2010-03-22 20:23:54,219021
21444,2011003225267462,1487992,2010-03-22 20:36:54,904089
21445,2011003225267462,1487992,2010-03-22 20:38:07,1494674
21446,2011003225267462,1487992,2010-03-22 20:39:08,1487261
21447,2011003225267462,1487992,2010-03-22 21:05:34,514250
21448,2011003225267462,1487992,2010-03-22 21:06:05,905919


In [18]:
final = ck.groupby(['original_status_id']).size()

In [19]:
final

original_status_id
211110210635991         148
211110620994663         215
2011007036040908        112
2111105204456415        656
2111105241247925        321
2111106244143052        203
2111107074596824        104
3336919622933457        178
3338502289692296        938
3339414328136339       8236
3340118641607783        254
3340209238447632        445
3340995544068457        294
3341107061397869        220
3341157665232491        409
3341753902473770        118
3341871566564264        191
3343026374817560        185
3343854785090350        375
3344913150613687        214
3345951588768390        381
3346738840959532        539
3347492286764509        208
3348028314070548        449
3348183368119710        175
3349206908266409        152
3349241598704866        103
3350025226504161        111
3350640477034147        283
3351362431925339        167
                       ... 
3511599434599056        347
3512258891991404        126
3512601373620544        177
3513830531488207        202
3

In [3]:
with  open('../data/micro/messaged-1k', 'rb') as mfile:
    message_dict = load(mfile)
with open('../data/micro/user_dict-1k', 'rb') as ufile:
    user_dict = load(ufile)

In [20]:
suffix = '1k'
with open('../data/micro/beta-{}'.format(suffix),'rb') as f:
    Beta = load(f)
with open('../data/micro/gamma-{}'.format(suffix),'rb') as f:
    Gamma = load(f)

In [21]:
Beta #followern | friendn | inflow | avg_inflow | avg_outflow | outflow

array([ 0.30747625,  0.02589092, -0.18649033, -0.00318369,  0.01340875,
       -0.01608643])

In [4]:
def map_index(df, col_list, dict_list):  # col is string array of column names, dict is a dictionary array
    df_mapped = df.copy()
    for i in range(len(col_list)):
        col = col_list[i]
        d = dict_list[i]
        # only apply if x exists as key , else return None

        df_mapped[col] = df_mapped[col].apply(lambda x: d.get(x))
        df_mapped = df_mapped.dropna(subset=[col, ])
    return df_mapped.set_index(col_list)

In [5]:
cascade_mapped = map_index(cascade, ['original_status_id', 'original_user_id'], [message_dict, user_dict])
cascade_list = cascade_mapped.reset_index().original_status_id.unique()
messagen = len(message_dict)
usern = len(user_dict) 

In [10]:
cascade_list #missing 109 143

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 18

In [15]:
len(user_dict)

820

In [6]:
def get_delta_series(group):
    # return Series
    s = pd.Series((group['time'] - group['time'].min()).apply(lambda dt: dt / timedelta(minutes=1)))
    return s


def to_sparse(t, m, n):  # create sparse matrix from multi-index
    i, j = list(zip(*t.index))
    data = t.values
    return sparse.coo_matrix((data, (i, j)), shape=(m, n))

In [7]:
def get_delta_count(group):
    return pd.DataFrame({
            'delta':(group['time'] - group['time'].min()).apply(lambda dt: dt / timedelta(minutes=1)),
            'original_status_id':group['original_status_id'],
            'original_user_id':group['original_user_id'],
            #'index':np.arange(len(group.index)),
        })

    

In [8]:
 C = cascade_mapped.reset_index().groupby(['original_status_id', 'original_user_id']).apply(get_delta_count)

In [9]:
C.head()

,delta,original_status_id,original_user_id
0,0.000000,0,160.0
1,33.500000,0,160.0
2,33.800000,0,160.0
3,0.000000,1,392.0
4,1.316667,1,392.0


In [10]:
C.original_user_id.nunique()

820

In [18]:
#how many subcascades one original_status contains 
# max 1
C[['original_status_id','original_user_id']].drop_duplicates().groupby('original_status_id').size().max()

1

In [9]:
C[['original_status_id','original_user_id']].drop_duplicates().groupby('original_user_id').size().max()

18

In [10]:
s = C.groupby(['original_status_id','original_user_id']).size()

In [13]:
s.describe()

count     989.000000
mean      159.663296
std       465.732770
min         2.000000
25%         7.000000
50%        27.000000
75%       111.000000
max      8236.000000
dtype: float64

In [12]:
s.min()

2

In [11]:
C.original_user_id = C.original_user_id.astype('int64')
C = C[C['delta']!=0]

In [12]:
C.head()

,delta,original_status_id,original_user_id
1,33.500000,0,160
2,33.800000,0,160
4,1.316667,1,392
5,8.166667,1,392
6,8.950000,1,392


In [17]:
C[C['original_user_id']==3] #some user_ids only have 1 time delta. Lambda goes to 0.

,delta,original_status_id,original_user_id
17543,0.516667,108,3


In [15]:
def prepare(user_data, cascade_mapped, messagen, usern):
    user_data.set_index('user_id', inplace=True)
    user_matrix = user_data.as_matrix()
    cascade = cascade_mapped.reset_index().dropna()
    # cascade.sort_values(['original_status_id', 'original_user_id', 'time'], axis=0, inplace=True)
    # M is the number of reposts for each <cascade, original_user> pair
    m = cascade.groupby(['original_status_id', 'original_user_id']).size()
    # m_sparse = to_sparse(m, messagen, usern)  # m is sparse
    
    C = cascade.groupby(['original_status_id', 'original_user_id']).apply(get_delta_count)#DataFrame 
    # C = cascade.groupby(['original_status_id', 'original_user_id']).apply(get_delta_series)
    # C.index = C.index.droplevel(2)  # drop the original index
    C.original_user_id = C.original_user_id.astype('int64')
    C = C[C['delta']!=0]

    return [user_matrix, m, C]

In [16]:
[user_matrix, m, C] = prepare(user_data, cascade_mapped, messagen, usern)

In [17]:
K = np.random.rand(usern, 1) + 0.5  # np.ones((usern, 1))
Lambda = np.random.rand(usern, 1) + 0.5
Beta = np.random.rand(6, 1)
Gamma = np.random.rand(6,1)
Alpha = [6 * 10 ** (-5), 8 * 10 ** (-6)]  # according to the paper
Mu = 10
Yita = 10

In [18]:
def get_C_row(C, usern, coln, i):
    tmp = pd.DataFrame(C.loc[i, :]).reset_index().drop('original_status_id', axis=1).set_index(['original_user_id'],
                                                                                               append=True)
    tmp.index = tmp.index.swaplevel(i=0, j=1)
    c_sparse = to_sparse(tmp['time'], usern, coln)
    return c_sparse.toarray()


def to_col(l):
    return np.ndarray(buffer=l, shape=(l.shape[0], 1))

In [20]:
def G1(Lambda, K, C, m, cascaden, usern):
    logL = 0
    for i in cascaden:
        coln = m.getrow(i).sum() - 1
        c_array = get_C_row(C, usern, coln, i)
        K_full = np.hstack((K,) * coln)
        p = to_col(np.power(c_array, K_full).sum(axis=1))
        a = m.getrow(i).transpose().multiply(np.log(K)).sum()
        b = (to_col(np.log(c_array + 1).sum(axis=1)) * (K - 1)).sum()
        c = m.getrow(i).transpose().multiply(K).multiply(np.log(Lambda)).sum()
        d = (np.power(Lambda, K * (-1)) * p).sum()
        logL = logL + a + b - c - d
    return -logL

In [15]:
from math import log,pow

In [19]:
def G1(Lambda,K,t1,t2,cascaden):
    logL =0
    for i in cascaden:
        if len(t1.loc[i].shape)==1:
            u = t1.loc[i].original_user_id
            time = t2.loc[i].loc[u].delta
            userK = K[u]
            userLambda = Lambda[u]
            logL += log(userK)+log(time)*(userK-1)-userK*log(userLambda)-pow(userLambda,userK*(-1))*pow(time,userK)
        else:
            users = t1.loc[i].original_user_id.drop_duplicates().as_matrix().tolist()
            for u in users:
                time = t2.loc[i].loc[u].delta.as_matrix()
                userK = K[u]
                userLambda = Lambda[u]
                m = time.shape[0]
                a = m * log(userK)
                b = np.log(time).sum()*(userK-1)
                c = m*userK*log(userLambda)
                d = pow(userLambda,userK*(-1))*np.power(time,userK).sum()
                logL += a+b-c-d
    return -logL

In [20]:
def G2(Lambda, Beta, user_data, Alpha, usern):
    loss = np.log(Lambda) - (np.log(user_data).dot(Beta))  # user_data should have no zeroes
    return 1 / (2 * usern) * np.linalg.norm(loss) ** 2 + Alpha[0] * np.abs(Beta).sum()  # L1 norm


def G3(K, Gamma, user_data, Alpha, usern):
    loss = np.log(K) - (np.log(user_data).dot(Gamma))
    return 1 / (2 * usern) * np.linalg.norm(loss)**2 + Alpha[1] * np.abs(Gamma).sum()  # L1 norm

In [21]:
def costLambda(Lambda, cascaden, usern, t1,t2, user_data, Alpha, Mu, K, Beta):
    J = G1(Lambda, K, t1,t2,cascaden) + Mu * G2(Lambda, Beta, user_data, Alpha, usern)
    return J

In [107]:
%timeit G1(Lambda,K,C,cascade_list,usern)

/home/zoey/anaconda3/envs/topics/lib/python3.5/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/zoey/anaconda3/envs/topics/lib/python3.5/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1 loop, best of 3: 2.46 s per loop


In [22]:
def derLambda(Lambda, cascaden, usern, t1,t2,user_data, Alpha, Mu, K, Beta):
    derG1 = np.zeros(Lambda.shape)  # col vector
    for i in cascaden:
        if len(t1.loc[i].shape) == 1:
            u = t1.loc[i].original_user_id
            time = t2.loc[i].loc[u].delta
            userK = K[u]
            userLambda = Lambda[u]
            p = pow(time, userK)
            a = userK / userLambda
            b = pow(userLambda, userK*(-1) - 1) * p * userK
            derG1[u] += a - b
        else:
            users = t1.loc[i].original_user_id.drop_duplicates().as_matrix().tolist()
            for u in users:
                time = t2.loc[i].loc[u].delta.as_matrix()
                userK = K[u]
                userLambda = Lambda[u]
                m = time.shape[0]
                p = np.power(time,userK).sum()
                a = m*userK/userLambda
                b = pow(userLambda,userK*(-1)-1) *p *userK
                derG1[u] += a-b

    loss = np.log(Lambda) - np.log(user_data).dot(Beta).reshape(Lambda.shape)
    derG2 = 1 / usern * (loss / Lambda)
    result = derG1 + derG2 * Mu
    return result

In [32]:
def derG1(Lambda,K,t1,t2,cascaden):#Lambda
    derG1 = np.zeros(Lambda.shape)  # col vector
    for i in cascaden:
        if len(t1.loc[i].shape) == 1:
            u = t1.loc[i].original_user_id
            time = t2.loc[i].loc[u].delta
            userK = K[u]
            userLambda = Lambda[u]
            p = pow(time, userK)
            a = userK / userLambda
            b = pow(userLambda, userK*(-1) - 1) * p * userK
            derG1[u] += a - b
        else:
            users = t1.loc[i].original_user_id.drop_duplicates().as_matrix().tolist()
            for u in users:
                time = t2.loc[i].loc[u].delta.as_matrix()
                userK = K[u]
                userLambda = Lambda[u]
                m = time.shape[0]
                p = np.power(time,userK).sum()
                a = m*userK/userLambda
                b = pow(userLambda,userK*(-1)-1) *p *userK
                derG1[u] += a-b
    return derG1


In [52]:
def derG1K(K,Lambda,t1,t2,cascaden):
    derG1 = np.zeros(K.shape)
    for i in cascaden:
        if len(t1.loc[i].shape) == 1:
            u = t1.loc[i].original_user_id
            time = t2.loc[i].loc[u].delta
            userK = K[u]
            userLambda = Lambda[u]
            p = pow(time, userK)
            q = pow(time, userK) * log(time)
            derG1[u] += - 1/ userK - log(time) + log(userLambda) - pow(userLambda, userK * (-1)) * log(userK) * p \
                + pow(userLambda, userK * (-1)) * q

    else:
            users = t1.loc[i].original_user_id.drop_duplicates().as_matrix().tolist()
            for u in users:
                time = t2.loc[i].loc[u].delta.as_matrix()
                print(time)
                userK = K[u]
                userLambda = Lambda[u]
                m = time.shape[0]
                p = np.power(time,userK).sum()
                q = (np.power(time,userK) * np.log(time)).sum()
                derG1[u] += - m/userK - np.log(time).sum() + m*log(userLambda) - pow(userLambda,userK*(-1)) *log(userLambda)*p\
                + pow(userLambda,userK*(-1))*q
    return derG1

In [43]:
def derG2(Lambda,Beta,user_data,Alpha,usern):
    loss = np.log(Lambda) - np.log(user_data).dot(Beta).reshape(Lambda.shape)
    derG2 = 1 / usern * (loss / Lambda)
    return derG2

In [42]:
from scipy.optimize import check_grad

In [27]:
t1 = C.set_index('original_status_id')
t2 = C.set_index(['original_status_id', 'original_user_id'])

In [56]:
cascade_list_test = cascade_list[1:2]

In [31]:
args = [cascade_list, usern, t1,t2, user_matrix, Alpha, Mu, K, Beta]
L = Lambda.reshape(usern,)
check_grad(costLambda,derLambda,L,*args)

/home/zoey/anaconda3/envs/topics/lib/python3.5/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/zoey/anaconda3/envs/topics/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/zoey/anaconda3/envs/topics/lib/python3.5/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


28253.987890343116

In [30]:
from math import log,pow

In [40]:
args = [K,t1,t2, cascade_list_test]
L = Lambda.reshape(usern,)
check_grad(G1,derG1,L,*args)

0.0022037347080186009

In [45]:
args = [Beta,user_matrix,Alpha,usern]
L = Lambda.reshape(usern,)
check_grad(G2,derG2,L,*args)

207.03564793483832

In [46]:
user_matrix.shape

(820, 6)

In [54]:
def G1K(K,Lambda,t1,t2,cascaden):
    logL =0
    for i in cascaden:
        if len(t1.loc[i].shape)==1:
            u = t1.loc[i].original_user_id
            time = t2.loc[i].loc[u].delta
            userK = K[u]
            userLambda = Lambda[u]
            logL += log(userK)+log(time)*(userK-1)-userK*log(userLambda)-pow(userLambda,userK*(-1))*pow(time,userK)
        else:
            users = t1.loc[i].original_user_id.drop_duplicates().as_matrix().tolist()
            for u in users:
                time = t2.loc[i].loc[u].delta.as_matrix()
                userK = K[u]
                userLambda = Lambda[u]
                m = time.shape[0]
                a = m * log(userK)
                b = np.log(time).sum()*(userK-1)
                c = m*userK*log(userLambda)
                d = pow(userLambda,userK*(-1))*np.power(time,userK).sum()
                logL += a+b-c-d
    return -logL

In [57]:
args = [Lambda,t1,t2,cascade_list_test]
K_test = K.reshape(usern,)
check_grad(G1K,derG1K,K_test,*args)

[   1.31666667    8.16666667    8.95          9.98333333   15.23333333
   15.8          24.51666667   40.55         41.2          58.28333333
   79.1         122.53333333  123.5         408.13333333  431.21666667
  578.75        596.35      ]


0.0013556447156588547

In [ ]:
#errors accumulate as the length of time grows 